# 2023-09-23 Demo: Machine Learning and Optimisation

In [1]:
# Install pyomo and amply
!pip install -q pyomo amplpy pandas numpy scikit-learn

In [2]:
# # Initialize AMPL 
# from amplpy import AMPL, tools

# ampl = tools.ampl_notebook(
#     modules=["coin", "highs", "gokestrel"], # modules to install
#     g=globals()) # instantiate AMPL object and register magics

In [3]:
import numpy as np
import pandas as pd
from amplpy import modules
import pyomo.environ as pyo
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression, Ridge

# Load Dataset

In [4]:
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

In [5]:
def solver_regression(X, y, solver, fit_intercept=True):
    
    # Intialise the solver
    solver = pyo.SolverFactory(modules.find(solver), solve_io="nl")

    # Set dimensions of dataset
    N, d = X.shape

    if fit_intercept:
        # Add another column of constants to be the bias/intercept term
        X_bias = np.hstack([np.ones((N, 1)), X])

    # Initialize Optimisation Model
    model = pyo.ConcreteModel()

    # Define decision variables (linear regression coefficients and intercept)
    model.w = pyo.Var(range(d+1), domain=pyo.Reals)

    # Objective
    model.obj = pyo.Objective(
        expr=sum((y[i] - sum(X_bias[i, j] * model.w[j] for j in range(d+1)))**2 for i in range(N)),
        sense=pyo.minimize
    )
    
    # Solve the model
    results = solver.solve(model)
    
    # Print the results
    if results.solver.status == pyo.SolverStatus.ok and results.solver.termination_condition == pyo.TerminationCondition.optimal:
        # Extracting coefficients
        coefficients = [round(model.w[i].value, 2) for i in range(d+1)]

        print("Solved! 🎉")
        print("Solver Coefficients:", coefficients)

        # Print regression formula
        formula = "y = {}".format(coefficients[0]) + " + "  + " + ".join("{}*x{}".format(coeff, idx) for idx, coeff in enumerate(coefficients[1:], 1))
        print("Regression Formula:", formula)

    else:
        print("Solver did not converge!")

    return coefficients

In [6]:
solver_coeffs = solver_regression(X, y, "ipopt")

Solved! 🎉
Solver Coefficients: [152.13, -10.01, -239.82, 519.85, 324.38, -792.18, 476.74, 101.04, 177.06, 751.27, 67.63]
Regression Formula: y = 152.13 + -10.01*x1 + -239.82*x2 + 519.85*x3 + 324.38*x4 + -792.18*x5 + 476.74*x6 + 101.04*x7 + 177.06*x8 + 751.27*x9 + 67.63*x10


In [7]:
# Fit regression model using scikit-learn for comparison
reg = LinearRegression(fit_intercept=True)
reg.fit(X, y)
sklearn_coeffs = [round(reg.intercept_, 2)] + [round(coef, 2) for coef in reg.coef_]
print("\nScikit-learn Coefficients:", sklearn_coeffs)


Scikit-learn Coefficients: [152.13, -10.01, -239.82, 519.85, 324.38, -792.18, 476.74, 101.04, 177.06, 751.27, 67.63]


In [8]:
print(solver_coeffs == sklearn_coeffs)

True


# Model 2: Robust Linear Regression (add L2 ridge penalty)

In [9]:
def solver_ridge(X, y, solver, alpha=1, fit_intercept=True):

    # Intialise the solver
    solver = pyo.SolverFactory(modules.find(solver), solve_io="nl")

    # Set dimensions of dataset
    N, d = X.shape

    if fit_intercept:
        # Add another column of constants to be the bias/intercept term
        X_bias = np.hstack([np.ones((N, 1)), X])

    # Initialize Optimisation Model
    model = pyo.ConcreteModel()

    # Define decision variables (linear regression coefficients and intercept)
    model.w = pyo.Var(range(d+1), domain=pyo.Reals)

    # Objective
    model.obj = pyo.Objective(
        expr=sum((y[i] - sum(X_bias[i, j] * model.w[j] for j in range(d + 1)))**2 for i in range(N))
        + alpha * sum(model.w[j] ** 2 for j in range(1, d + 1)),  # Exclude the intercept term
      sense=pyo.minimize
    )

    # Solve the model
    results = solver.solve(model)

    # Print the results
    if results.solver.status == pyo.SolverStatus.ok and results.solver.termination_condition == pyo.TerminationCondition.optimal:
        # Extracting coefficients
        coefficients = [round(model.w[i].value, 2) for i in range(d+1)]

        print("Solved! 🎉")
        print("Solver Coefficients:", coefficients)

        # Print regression formula
        formula = "y = {}".format(coefficients[0]) + " + " + " + ".join("{}*x{}".format(coeff, idx) for idx, coeff in enumerate(coefficients[1:], 1))
        print("")
        print("Regression Formula:", formula)

    else:
        print("Solver did not converge!")

    return coefficients

In [10]:
solver_ridge_coeffs = solver_ridge(X, y, "ipopt", alpha=10)

Solved! 🎉
Solver Coefficients: [152.13, 19.81, -0.92, 75.42, 55.03, 19.92, 13.95, -47.55, 48.26, 70.14, 44.21]

Regression Formula: y = 152.13 + 19.81*x1 + -0.92*x2 + 75.42*x3 + 55.03*x4 + 19.92*x5 + 13.95*x6 + -47.55*x7 + 48.26*x8 + 70.14*x9 + 44.21*x10


In [11]:
# Fit ridge model using scikit-learn for comparison
ridge_reg = Ridge(alpha=10, fit_intercept=True)
ridge_reg.fit(X, y)
sklearn_ridge_coeffs = [round(ridge_reg.intercept_, 2)] + [round(coef, 2) for coef in ridge_reg.coef_]
print("\nScikit-learn Coefficients:", sklearn_ridge_coeffs)


Scikit-learn Coefficients: [152.13, 19.81, -0.92, 75.42, 55.03, 19.92, 13.95, -47.55, 48.26, 70.14, 44.21]


In [12]:
print(solver_ridge_coeffs == sklearn_ridge_coeffs)

True


# Model 3: Sparse Robust Linear Regression

In [13]:
def solver_sparse_ridge(X, y, max_non_zero, solver, alpha=1, fit_intercept=True):
    
    # Intialise the solver
    solver = pyo.SolverFactory(solver)

    # Set dimensions of dataset
    N, d = X.shape

    # Add another column of constants to be the bias/intercept term
    if fit_intercept:
        X = np.hstack([np.ones((N, 1)), X])

    K = max_non_zero
    M = 1e6  # Big M value

    # Create the model
    model = pyo.ConcreteModel()

    # Variables
    model.beta = pyo.Var(range(d+1), within=pyo.Reals)  # Regression coefficients
    model.z = pyo.Var(range(d+1), within=pyo.Binary)  # Binary variables to enforce sparsity

    # Objective function: Minimize the sum of squared errors + L2 penalty
    model.obj = pyo.Objective(
        expr=sum((y[i] - sum(X[i][j] * model.beta[j] for j in range(d+1)))**2 for i in range(N))
            + alpha * sum(model.beta[j]**2 for j in range(1, d+1)),  # L2 penalty term starts from 1
        sense=pyo.minimize
    )

    # Constraints
    model.sparse_const = pyo.Constraint(expr=sum(model.z[j] for j in range(1, d+1)) <= K)  # Excludes the constant

    # Big M constraints to enforce sparsity
    model.positive_bigM = pyo.ConstraintList()
    model.negative_bigM = pyo.ConstraintList()
    for j in range(1, d+1):  # Starts from 1 to exclude the constant
        model.positive_bigM.add(expr=model.beta[j] <= M * model.z[j])
        model.negative_bigM.add(expr=model.beta[j] >= -M * model.z[j])

    # Solve the model
    results = solver.solve(model)

    # Print the results
    if results.solver.status == pyo.SolverStatus.ok and results.solver.termination_condition == pyo.TerminationCondition.optimal:
        # Extracting coefficients
        coefficients = [round(model.beta[i].value, 2) for i in range(d)]

        print("Solved! 🎉")
        print("Solver Coefficients:", coefficients)

        # Print regression formula
        formula = "y = {}".format(coefficients[0]) + " + " + " + ".join("{}*x{}".format(coeff, idx) for idx, coeff in enumerate(coefficients[1:], 1))
        print("")
        print("Regression Formula:", formula)

    else:
        print("Solver did not converge!")

    return coefficients

In [14]:
sparse_ridge_coeffs = solver_sparse_ridge(X, y, 2, "bonmin")

Solved! 🎉
Solver Coefficients: [152.13, -0.0, 0.0, 392.04, -0.0, 0.0, -0.0, -0.0, -0.0, 370.61]

Regression Formula: y = 152.13 + -0.0*x1 + 0.0*x2 + 392.04*x3 + -0.0*x4 + 0.0*x5 + -0.0*x6 + -0.0*x7 + -0.0*x8 + 370.61*x9


In [15]:
print(f"Number of non-zero coefficients: {np.sum(np.array(sparse_ridge_coeffs[1:]) >  0)}")

Number of non-zero coefficients: 2
